# Polynomial features

The modelling tools included in `ISLP` allow for
construction of orthogonal polynomials of features.

Force rebuild

In [1]:
import numpy as np
from ISLP import load_data
from ISLP.models import ModelSpec, poly

In [2]:
Carseats = load_data('Carseats')
Carseats.columns

Index(['Sales', 'CompPrice', 'Income', 'Advertising', 'Population', 'Price',
       'ShelveLoc', 'Age', 'Education', 'Urban', 'US'],
      dtype='object')

Let's make a term representing a quartic effect for `Population`.

In [3]:
quartic = poly('Population', 4)

The object `quartic` does not refer to any data yet, it must be included in a `ModelSpec` object
and fit using the `fit` method.

In [4]:
design = ModelSpec([quartic], intercept=False)
ISLP_features = design.fit_transform(Carseats)
ISLP_features.columns

Index(['poly(Population, degree=4)[0]', 'poly(Population, degree=4)[1]',
       'poly(Population, degree=4)[2]', 'poly(Population, degree=4)[3]'],
      dtype='object')

## Compare to `R`

We can compare our polynomials to a similar function in `R`

In [5]:
%load_ext rpy2.ipython

We'll recompute these features using `poly` in `R`.

In [6]:
%%R -i Carseats -o R_features
R_features = poly(Carseats$Population, 4)

In [7]:
np.linalg.norm(ISLP_features - R_features)

3.95855733196009e-15

## Underlying model

If we look at `quartic`, we see it is a `Variable`, i.e. it can be used to produce a set of columns
in a design matrix when it is a term used in creating the `ModelSpec`.

Its encoder is `Poly(degree=4)`. This is a special `sklearn` transform that expects a single column
in its `fit()` method and constructs a matrix of corresponding orthogonal polynomials.

The spline helpers `ns` and `bs` as well as `pca` follow a similar structure.

In [8]:
quartic

Variable(variables=('Population',), name='poly(Population, degree=4)', encoder=Poly(degree=4), use_transform=True, pure_columns=False, override_encoder_colnames=True)

## Raw polynomials

One can compute raw polynomials (which results in a less well-conditioned design matrix) of course.

In [9]:
quartic_raw = poly('Population', degree=4, raw=True)

Let's compare the features again.

In [10]:
design = ModelSpec([quartic_raw], intercept=False)
raw_features = design.fit_transform(Carseats)

In [11]:
%%R -i Carseats -o R_features
R_features = poly(Carseats$Population, 4, raw=TRUE)

In [12]:
np.linalg.norm(raw_features - R_features)

0.0

## Intercept

Looking at `py_features` we see it contains columns: `[Population**i for i in range(1, 4)]`. That is, 
it doesn't contain an intercept, the order 0 term. This can be include with `intercept=True`

In [13]:
quartic_int = poly('Population', degree=4, raw=True, intercept=True)
design = ModelSpec([quartic_int], intercept=False)
intercept_features = design.fit_transform(Carseats)

In [15]:
np.linalg.norm(intercept_features.iloc[:,1:] - R_features)

0.0